In [1]:
import pandas as pd
import torch
%matplotlib inline

import time
import numpy as np

from torchvision import datasets
from torchvision import transforms

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

ModuleNotFoundError: No module named 'torchvision'

In [ ]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Note transforms.ToTensor() scales input images to 0-1 range
train_dataset = datasets.MNIST(root='data', 
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='data', 
                              train=False, 
                              transform=transforms.ToTensor())

train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    # (128, 1, 28, 28) = (batch_size, color, height, width)
    print('Image batch dimensions:', images.shape)
    # (128, 1) = (batch_size, 1)
    print('Image label dimensions:', labels.shape)
    break

In [ ]:
class NLP(torch.nn.Module):
    def __init__(self, n_features, n_hidden, n_classes):
        super().__init__()
        
        self.n_classes = n_classes
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(n_features, n_hidden),
            nn.Sigmoid(),
            nn.Linear(n_hidden, n_classes),
        )
        
    def forward(self, x):
        logits = self.model(x)
        probas = torch.softmax(logits, dim=1)
        return logits, probas

# negative log-liklihood loss
def nll_loss(x, y):
    y = torch.from_numpy(np.take(np.eye(10), y, axis=0))
    return torch.sum(x * y)

def get_accuracy(logits, y):
    pred = torch.argmax(logits, axis=1)
    corr_pred = torch.sum(pred == y)
    return corr_pred / len(y)

def nll_loss_test(model, dataloader):
    curr_loss = 0
    
    model.eval()  # disable norms and dropout
    with torch.no_grad():  # disable backprop
        
        for i, (imgs, labels) in enumerate(dataloader):
            imgs.to(DEVICE)
            labels.to(DEVICE)
            
            logits, probas = model(imgs)
            curr_loss += nll_loss(probas, labels)
            
        return curr_loss / (i + 1)

In [ ]:
BATCH_SIZE = 256
EPOCHS = 500

model = NLP(n_features=28*28, n_hidden=64, n_classes=10).to(DEVICE)
opt = torch.optim.SGD(model.parameters(), lr=.1)

## Training Phase
for epoch in range(EPOCHS):
    model.train()
    
    cum_loss = 0
    corr_pred = 0
    
    for i, (imgs, labels) in enumerate(train_loader):
        
        imgs.to(DEVICE)
        labels.to(DEVICE)
        
        logits, probas = model(imgs)
        loss = nll_loss(probas, labels)
        
        cum_loss += loss
        corr_pred += get_accuracy(logits, labels)
        
        opt.zero_grad()
        loss.backward()
        opt.step()
   
    if not epoch % (EPOCHS // 100):
        print(f"Epoch: {epoch:03d} | Train. Loss: {cum_loss / (i + 1):.4f} | Train Acc: {corr_pred / len(train_loader) * 100:.2f}% \
          | Test Loss: {nll_loss_test(model, test_loader):.4f}")